In [1]:
import pandas as pd
import numpy as np

In [4]:
def uptime_duration(data, value_threshold, verbose=False):
    #FIXME how are you dealing with selecting the column
    # assumption: we are only summing the uptimes that are completely within our data selection
    grid_on = data['Voltage V'] > value_threshold

    # by doing a pair-wise subtraction we get the changes in the grid
    grid_events = grid_on.astype(int).diff(1).dropna()
    grid_events = grid_events[grid_events != 0.0]

    # we want the first event to be a grid on event
    if grid_events.iloc[0] == -1:
        grid_events = grid_events[1:]
    # we want the last event to be a grid off event
    if grid_events.iloc[-1] == 1:
        grid_events = grid_events[:-1]

    # by subtracting the datetime indices for the events, we get the durations
    grid_goes_on = grid_events[grid_events == 1]
    grid_goes_off = grid_events[grid_events == -1]
    durations = (grid_goes_off.index - grid_goes_on.index) / np.timedelta64(1,'h')

    durations = pd.DataFrame(data = {'durations':durations}, index = grid_goes_on.index)
    return float(durations.sum())

In [5]:
value_threshold = 100

plug_load = pd.read_csv('Plug_Load_Logger7916 (1).csv', skiprows=1, index_col=1, parse_dates=True)
uptime_duration(plug_load, value_threshold)

durations    348.566667
dtype: float64